# 🧪 GPTQ From Scratch: Optimal Brain Quantization (2023)

[!["Open In Colab"](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/model-size-reduction/blob/main/chronology/gptq_demo.ipynb)

## 📖 The Theory: OBQ & The Hessian Matrix

GPTQ (Generalized Post-Training Quantization) is a high-performance 4-bit quantization method. It is based on **Optimal Brain Quantization (OBQ)**, which aims to minimize the error between the original weight and the quantized weight, weighted by the sensitivity of the layer.

### The Objective Function
We want to find $W_{quant}$ that minimizes squared error, but not all weights are equal. Some weights "hurt" more when rounded. This sensitivity is captured by the **Hessian matrix** $H$ ($H = 2XX^T$).

$$\min_{W_{quant}} ||W X - W_{quant} X||_2^2 \approx \min_{W_{quant}} \sum_{i} (w_i - w_{quant,i})^2 H_{ii}$$

### The Greedy Update
GPTQ quantizes weights one by one. After quantizing weight $w_i$, it adjusts the *remaining* weights to compensate for the introduced error:

$$\delta w = -(w_i - round(w_i)) \cdot \frac{1}{[H^{-1}]_{ii}} \cdot [H^{-1}]_{:,i}$$

Where $H^{-1}$ is the inverse Hessian. This "compensatory" step is what makes GPTQ so accurate at 4-bits.

---

In [ ]:
import torch
import torch.nn as nn

class GPTQManual:
    def __init__(self, layer):
        self.layer = layer
        self.dev = layer.weight.device
        W = layer.weight.data.clone()
        self.rows = W.shape[0]
        self.columns = W.shape[1]
        self.H = torch.zeros((self.columns, self.columns), device=self.dev)
        self.nsamples = 0

    def add_batch(self, inp):
        # Accumulate Hessian information from input activations
        if len(inp.shape) == 2:
            inp = inp.unsqueeze(0)
        tmp = inp.shape[0]
        if isinstance(self.layer, nn.Linear):
            if len(inp.shape) == 3:
                inp = inp.reshape((-1, inp.shape[-1]))
            inp = inp.t()
        self.H *= self.nsamples / (self.nsamples + tmp)
        self.nsamples += tmp
        inp = inp.float()
        self.H += 2 / self.nsamples * torch.matmul(inp, inp.t())

    def quantize(self, bits=4):
        W = self.layer.weight.data.clone().float()
        H = self.H
        
        # Cholesky decomposition to stably invert the Hessian
        dead = torch.diag(H) == 0
        H[dead, dead] = 1
        W[:, dead] = 0
        
        # Regularization to make it invertible
        reg = 0.01 * torch.mean(torch.diag(H))
        H += reg * torch.eye(self.columns, device=self.dev)
        H_inv = torch.linalg.inv(H)
        
        # Manual Quantization Loop
        Q = torch.zeros_like(W)
        
        # For demonstration: Symmetric Min-Max Quantization
        scale = (torch.max(torch.abs(W)) / (2**(bits-1) - 1))
        
        print(f"Starting GPTQ loop for {self.columns} columns...")
        for i in range(self.columns):
            w = W[:, i]
            # 1. Quantize weight
            q = torch.round(w / scale).clamp(-(2**(bits-1)), 2**(bits-1)-1)
            q_val = q * scale
            Q[:, i] = q_val
            
            # 2. Error
            err = (w - q_val) / H_inv[i, i]
            
            # 3. Compensate remaining weights (the OBQ update)
            W[:, i:] -= err.unsqueeze(1) * H_inv[i, i:].unsqueeze(0)
            
        self.layer.weight.data = Q.to(self.layer.weight.dtype)
        print("Quantization Complete.")

In [ ]:
# Test implementation
layer = nn.Linear(128, 256)
original_weight = layer.weight.data.clone()

gptq = GPTQManual(layer)
# Simulate 10 batches of calibration data
for _ in range(10):
    calibration_data = torch.randn(1, 128)
    gptq.add_batch(calibration_data)

gptq.quantize(bits=4)

unique_vals = torch.unique(layer.weight.data).shape[0]
print(f"Unique values in weight matrix: {unique_vals} (Expected ~16 for 4-bit)")